In [1]:
from data.MFTComparedData.mft_dim import combine_moral_foundations, generate_mc_prompts

In [2]:
combined_moral_foundations = combine_moral_foundations()


In [3]:
combined_moral_foundations.keys()


dict_keys(['care', 'fairness', 'loyalty', 'authority', 'sanctity', 'liberty'])

In [6]:
# Save combined moral foundations to file
import json

with open('combined_moral_foundations.json', 'w') as f:
    json.dump(combined_moral_foundations, f, indent=2)

In [20]:
[pair[0] for pair in combined_moral_foundations4['care']]


['One should always come to the aid of a stranger in distress.',
 'Providing shelter and food to the homeless is our fundamental duty.',
 'We should speak out against bullying to protect vulnerable individuals.',
 'Inflicting physical harm on animals for entertainment is cruel and should be prohibited.',
 'Every child deserves emotional support and reassurance during hard times.',
 'Society should allocate resources to ensure that the sick receive proper medical care.',
 'We have a moral responsibility to report and prevent abuse when we see it.',
 'Comforting someone grieving a loss is a sign of basic human compassion.',
 'Offering support to coworkers under stress fosters a more humane work environment.',
 'Rescuing neglected pets and giving them care demonstrates moral empathy.',
 'Responding kindly when someone makes an honest mistake prevents unnecessary harm.',
 'Seeking help for mental health issues is important; we should encourage one another to do so.',
 'Volunteering at shel

In [12]:
prompts = generate_mc_prompts(combined_moral_foundations, instruct_model=True)

In [13]:
for p in prompts:
    print("-" * 50)
    print(f"Foundation: {p['foundation']}")
    print(p["prompt"], end="")  # 'end=""' to place the cursor right after "Answer: "
    # For demonstration, let's say we know the correct answer:
    print(f"(Correct: {p['correct_answer']})")

--------------------------------------------------
Foundation: care
Answer only with the letter of the answer no explanation only one letter.
Which statement best represents the moral dimension of 'care'?
  A) It’s acceptable to ignore a stranger’s suffering if it doesn’t affect you.
  B) One should always come to the aid of a stranger in distress.
Answer: (Correct: B)
--------------------------------------------------
Foundation: care
Answer only with the letter of the answer no explanation only one letter.
Which statement best represents the moral dimension of 'care'?
  A) Homelessness is their own problem; society has no obligation to intervene.
  B) Providing shelter and food to the homeless is our fundamental duty.
Answer: (Correct: B)
--------------------------------------------------
Foundation: care
Answer only with the letter of the answer no explanation only one letter.
Which statement best represents the moral dimension of 'care'?
  A) Bullying is a natural way for some peop

# Load Gemma-2-9b-it model to test if it is working with the data

In [11]:
import torch
from transformer_lens import HookedTransformer

# Load Gemma model using TransformerLens
model_name = "google/gemma-2-9b-it"
model = HookedTransformer.from_pretrained(
    model_name,
    device="cuda" if torch.cuda.is_available() else "cpu",
    dtype=torch.float16  # Use float16 for efficiency
)

print(f"Model loaded: {model_name}")
print(f"Number of parameters: {model.cfg.n_params/1e9:.1f}B")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/root/miniconda/envs/mech/lib/python3.10/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


Loaded pretrained model google/gemma-2-9b-it into HookedTransformer
Model loaded: google/gemma-2-9b-it
Number of parameters: 8.9B


In [36]:
# Process all prompts with padding and track last real token positions
max_len = max(len(model.to_tokens(p["prompt"] + "A")[0]) for p in prompts)

# Store the last real token position for each prompt
last_token_pos = torch.tensor([
    len(model.to_tokens(p["prompt"] + p["correct_answer"])[0]) 
    for p in prompts
], device=model.cfg.device)

clean_tokens = torch.stack([
    torch.nn.functional.pad(
        model.to_tokens(p["prompt"] + p["correct_answer"])[0],
        (0, max_len - len(model.to_tokens(p["prompt"] + p["correct_answer"])[0]))
    ) for p in prompts
])
corrupt_tokens = torch.stack([
    torch.nn.functional.pad(
        model.to_tokens(p["prompt"] + ("A" if p["correct_answer"] == "B" else "B"))[0],
        (0, max_len - len(model.to_tokens(p["prompt"] + ("A" if p["correct_answer"] == "B" else "B"))[0]))
    ) for p in prompts
])
answer_token_indices = torch.tensor([
    [model.to_single_token(p["correct_answer"]), 
     model.to_single_token("A" if p["correct_answer"] == "B" else "B")]
    for p in prompts
], device=model.cfg.device)

In [38]:
print(clean_tokens.shape)
print(corrupt_tokens.shape)
print(answer_token_indices.shape)
print(last_token_pos.shape)

torch.Size([240, 81])
torch.Size([240, 81])
torch.Size([240, 2])
torch.Size([240])


In [53]:
print(model.to_string(clean_tokens[0][last_token_pos[0]-1]))
print(model.to_string(corrupt_tokens[0][last_token_pos[0]-1]))
print(model.to_string(answer_token_indices[0]))


 B
 A
BA


In [55]:
def get_logit_diff(logits, answer_tokens):
    if len(logits.shape)==3:
        # Get final logits only
        logits = logits[:, -1, :]
    correct_logits = logits.gather(1, answer_tokens[0].unsqueeze(0).unsqueeze(0))
    incorrect_logits = logits.gather(1, answer_tokens[1].unsqueeze(0).unsqueeze(0))
    return (correct_logits - incorrect_logits).item()

# Process prompts one by one and collect results
clean_logit_diffs = []
corrupted_logit_diffs = []

for i in range(len(prompts)):
    # Get single prompt tokens
    clean_prompt = clean_tokens[i:i+1]
    corrupt_prompt = corrupt_tokens[i:i+1]
    answer_tokens = answer_token_indices[i]
    
    # Run model on single prompt
    clean_logits, _ = model.run_with_cache(clean_prompt)
    corrupted_logits, _ = model.run_with_cache(corrupt_prompt)
    
    # Calculate logit differences
    clean_logit_diffs.append(get_logit_diff(clean_logits, answer_tokens))
    corrupted_logit_diffs.append(get_logit_diff(corrupted_logits, answer_tokens))

# Calculate means
clean_logit_diff = sum(clean_logit_diffs) / len(clean_logit_diffs)
corrupted_logit_diff = sum(corrupted_logit_diffs) / len(corrupted_logit_diffs)

print(f"Clean logit diff: {clean_logit_diff:.4f}")
print(f"Corrupted logit diff: {corrupted_logit_diff:.4f}")



Clean logit diff: 0.2308
Corrupted logit diff: 0.2432


## Load Phi-4 model to test if it is working with the data

In [3]:
from IPython.display import clear_output

In [4]:
import nnsight
from nnsight import CONFIG

In [5]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected+colab+notebook"
from nnsight import LanguageModel, util
from nnsight.tracing.Proxy import Proxy

In [7]:
model = LanguageModel("microsoft/phi-4", device_map="auto")
clear_output()

In [8]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(100352, 5120, padding_idx=100257)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (qkv_proj): Linear(in_features=5120, out_features=7680, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=5120, out_features=35840, bias=False)
          (down_proj): Linear(in_features=17920, out_features=5120, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((5120,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((5120,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((5120,), eps=1e-05

In [28]:
prompts[0]["prompt"]

"Answer only with the letter of the answer no explanation only one letter.\nWhich statement best represents the moral dimension of 'care'?\n  A) It’s acceptable to ignore a stranger’s suffering if it doesn’t affect you.\n  B) One should always come to the aid of a stranger in distress.\nAnswer: "

In [65]:
model.model.layers

ModuleList(
  (0-39): 40 x Phi3DecoderLayer(
    (self_attn): Phi3SdpaAttention(
      (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
      (qkv_proj): Linear(in_features=5120, out_features=7680, bias=False)
      (rotary_emb): Phi3RotaryEmbedding()
    )
    (mlp): Phi3MLP(
      (gate_up_proj): Linear(in_features=5120, out_features=35840, bias=False)
      (down_proj): Linear(in_features=17920, out_features=5120, bias=False)
      (activation_fn): SiLU()
    )
    (input_layernorm): Phi3RMSNorm((5120,), eps=1e-05)
    (resid_attn_dropout): Dropout(p=0.0, inplace=False)
    (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
    (post_attention_layernorm): Phi3RMSNorm((5120,), eps=1e-05)
  )
)

In [78]:
with model.trace("The Eiffel Tower is in the city of"):

    # Access the last layer using h[-1] as it's a ModuleList
    # Access the first index of .output as that's where the hidden states are.
    model.model.layers[-1].self_attn.output[0][:] = 0

    # Logits come out of model.lm_head and we apply argmax to get the predicted token ids.
    token_ids = model.lm_head.output.argmax(dim=-1).save()

print("\nToken IDs:", token_ids)

# Apply the tokenizer to decode the ids into words after the tracing context.
print("Prediction:", model.tokenizer.decode(token_ids[0][-1]))


Token IDs: tensor([[ 5575,  3168,   301, 22703,   374,  7559, 12366,  3363,   315, 12366]],
       device='cuda:2')
Prediction:  Paris


In [79]:
with model.trace("hello", invoker_args={"max_length":10}) as tracer:
  invoker = tracer.invoker

/root/miniconda/envs/mech/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2855: UserWarning:

`max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.



In [27]:
N_LAYERS = len(model.model.layers)
print(range(38, N_LAYERS))

print(prompts[0]["correct_answer"])

with model#with model.trace(prompts[0]["prompt"]):


range(38, 40)
B
